<!-- ![logo](miniconslogo.png) -->
<!-- <img src="miniconslogo.png" alt="minicons" style="width:150px;display: block; margin-left: auto; margin-right: auto;"/> -->

# Scorer Module

The scorer module elicits conditional probabilities from stimuli, given any model trained using Huggingface `transformers`

In [5]:
import torch

from minicons import scorer

## Model Instantiation

minicons can use any model from Huggingface Transformers

Here, we use GPT2

In [2]:
lm = scorer.IncrementalLMScorer('gpt2', 'cpu') # for GPU use cuda:<DEVICENAME>

Using pad_token, but it is not set yet.


## Demo: Grammatical Acceptability

Sentences collected from BLiMP (Warstadt et al., 2020)

In [55]:
acceptable_sentences = [
    'The keys to the cabinet are on the table.',
    'The cats annoy Tim.',
    "Rose wasn't disturbing Mark.",
    'Carlos said that Lori helped him.'
]

unacceptable_sentences = [
    'The keys to the cabinet is on the table.',
    'The cats annoys Tim.',
    "Rose wasn't boasting Mark",
    'Carlos said that Lori helped himself.'
]

### Batch-wise sentence log-probabilities per token:

In [33]:
acceptable_scores = lm.sequence_score(acceptable_sentences)
unacceptable_scores = lm.sequence_score(unacceptable_sentences)

list(zip(acceptable_sentences, acceptable_scores))

[('The keys to the cabinet are on the table.', -5.607510089874268),
 ('The cats annoy Tim.', -12.293123245239258),
 ("Rose wasn't disturbing Mark.", -9.35051441192627),
 ('Carlos said that Lori helped him.', -8.524112701416016)]

### Accuracy

Percentage of time model scores acceptable greater than unacceptable:

In [57]:
comparisons = torch.tensor(acceptable_scores) > torch.tensor(unacceptable_scores)
acc = comparisons.float().mean().item()

print(f"The model achieves an accuracy of {round(acc*100, 2)}%")

The model achieves an accuracy of 75.0%


## Token-wise log-probabilities, in batched manner:

In [36]:
lm.token_score(unacceptable_sentences)

[[('The', 0.0),
  ('keys', -9.825798034667969),
  ('to', -0.9178085327148438),
  ('the', -2.2056198120117188),
  ('cabinet', -8.424751281738281),
  ('is', -4.4639739990234375),
  ('on', -5.424781799316406),
  ('the', -0.416473388671875),
  ('table', -4.647712707519531),
  ('.', -1.681610107421875)],
 [('The', 0.0),
  ('cats', -10.47793197631836),
  ('annoy', -12.409324645996094),
  ('s', -5.624114990234375),
  ('Tim', -8.530014038085938),
  ('.', -2.160125732421875)],
 [('Rose', 0.0),
  ('wasn', -8.536266326904297),
  ("'t", -0.002288818359375),
  ('boasting', -11.13873291015625),
  ('Mark', -12.629707336425781)],
 [('Carl', 0.0),
  ('os', -6.095460891723633),
  ('said', -5.179496765136719),
  ('that', -2.370147705078125),
  ('Lori', -12.451545715332031),
  ('helped', -7.916389465332031),
  ('himself', -7.626190185546875),
  ('.', -4.432945251464844)]]

## Accuracy of BERT-base-uncased

On the same four sentence-pair stimuli:

In [59]:
lm = scorer.MaskedLMScorer('bert-base-uncased', 'cpu') # for GPU use cuda:<DEVICENAME>

acceptable_scores = lm.sequence_score(acceptable_sentences, base_two=True)
unacceptable_scores = lm.sequence_score(unacceptable_sentences, base_two=True)

comparisons = torch.tensor(acceptable_scores) > torch.tensor(unacceptable_scores)

comparisons = torch.tensor(acceptable_scores) > torch.tensor(unacceptable_scores)
acc = comparisons.float().mean().item()

print(f"The model achieves an accuracy of {round(acc*100, 2)}%")

The model achieves an accuracy of 75.0%


# cwe Module

The cwe (pronounced "sewey") module allows extraction of token/phrase representations at various layers of any huggingface `transformers` model

In [60]:
from minicons import cwe

In [63]:
lm = cwe.CWE('bert-base-uncased', 'cpu') # for GPU use cuda:<DEVICENAME>

## Input representation

`cwe` expects inputs to be formatted in the following way:

```py
[
  (sentence_1, word_1),
  (sentence_2, word_2),
  ....
  (sentence_n, word_n)
]
```

In [61]:
stimuli = [
    ['This aircraft works by jet propulsion.', 'aircraft'],
    ['His passion is making model aircraft.', 'aircraft'],
    ['The aircraft was flying in a northerly direction.', 'aircraft'],
    ['A small aircraft was obstructing the runway.', 'aircraft'],
    ['The aircraft is powered by three jet engines.', 'aircraft']
]

### Extract representations from any layer or a combination of layers

In [65]:
lm.extract_representation(stimuli, layer = 1)

tensor([[ 0.1875,  1.3159, -0.5299,  ..., -0.6241,  1.1589,  0.1093],
        [ 1.0992,  1.2651, -0.5388,  ..., -0.4278,  1.3408,  0.1456],
        [ 0.2353,  1.4479, -0.6632,  ..., -0.1825,  1.3137, -0.1166],
        [ 0.5227,  1.0542, -0.6658,  ..., -0.5368,  1.2150, -0.0836],
        [ 0.0175,  1.2684, -0.6548,  ..., -0.2252,  1.1569, -0.1668]])

In [67]:
lm.extract_representation(stimuli, layer = [1,6,5])

[tensor([[ 0.1875,  1.3159, -0.5299,  ..., -0.6241,  1.1589,  0.1093],
         [ 1.0992,  1.2651, -0.5388,  ..., -0.4278,  1.3408,  0.1456],
         [ 0.2353,  1.4479, -0.6632,  ..., -0.1825,  1.3137, -0.1166],
         [ 0.5227,  1.0542, -0.6658,  ..., -0.5368,  1.2150, -0.0836],
         [ 0.0175,  1.2684, -0.6548,  ..., -0.2252,  1.1569, -0.1668]]),
 tensor([[ 1.1305,  1.2379, -0.3605,  ..., -0.2164,  0.7634,  0.2290],
         [ 1.5314,  1.1103, -0.3012,  ...,  0.3013,  1.1243, -0.1035],
         [ 0.5519,  0.3571,  0.1852,  ..., -0.0317,  0.3467, -0.5793],
         [ 0.4403,  0.9158,  0.3346,  ..., -0.6312,  0.7955, -0.0562],
         [ 0.2733,  1.0285, -0.3388,  ...,  0.5740,  0.2786, -0.1991]]),
 tensor([[ 1.0789,  1.2996, -0.3051,  ..., -0.3042,  0.2242,  0.3323],
         [ 1.6647,  1.3595,  0.0095,  ...,  0.2924,  1.0470, -0.4102],
         [ 0.4315,  0.2476,  0.2358,  ..., -0.4528,  0.4465, -0.5881],
         [ 0.4200,  1.0115,  0.6303,  ..., -1.0683,  0.8520, -0.1810],
  